In [9]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
import time
import bs4
import re
import os 
import urllib.request, urllib.error, urllib.parse

In [10]:
# Read the dataset with the profile links
profile_urls = pd.read_csv('RSC reviews with profile ids.csv')

In [11]:
profile_urls = profile_urls[['product','profile']]

In [12]:
profile_urls = profile_urls.head(11)

In [13]:
profile_urls.head()

,product,profile
0,B078N8NR7G,https://www.amazon.ca/gp/profile/amzn1.account...
1,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...
2,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...
3,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...
4,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...


In [ ]:
for i in range(0,11):
    time.sleep(10) #Hold 10 seconds before the next scrape.
    num=str(i)
    newurl = profile_urls.loc[i,'profile']
    d = webdriver.Chrome(executable_path=os.path.abspath('chromedriver')) 
    d.get(newurl)
    d.execute_script('document.documentElement.scollTop=10000')# To make the webdriver scroll down to fetch as many as the reviews.
    time.sleep(5) # sleep again the let the page load
    name='profile'+num+'.txt' #The new file name. 
    file=open(name,'w') #Create a new file.
    file.write(d.page_source) #Write the html requested into it.
    file.close()
    d.close()#Close the google webpage that webdriver open for you, otherwise it will be crazy.

In [14]:
soup = {}
for i in range(0,11):
    num=str(i)
    name='profile'+num+'.txt'
    text=open(name,'r').read() #Read the text from the file.
    soup[i]=bs4.BeautifulSoup(text) #Create a beautifulsoup object using the txt we got.

In [405]:
# generating new columns: all related one reviewer 
for i in profile_urls.index.unique():
    tag0 = soup[i].find_all('div', class_='dashboard-desktop-stat-value')[0] 
    profile_urls.loc[i,'helpful_votes'] = tag0.find('span', class_='a-size-large a-color-base').get_text() 
    
    for tag in soup[i].find_all('div', class_='a-row a-spacing-none name-container'):    
        profile_urls.loc[i,'name'] = tag.find('span', class_='a-size-extra-large').get_text() 
    
    for tag in soup[i].find_all('div', class_='dashboard-desktop-stat-value'):    
        profile_urls.loc[i,'num_of_reviews'] = int(tag.find('span', class_='a-size-large a-color-base').get_text())
    
    profile_urls.loc[i,'num_of_reviews_count'] = len(soup[i].find_all('div', class_='a-section profile-at-content'))
    
    verified = []
    for tag in soup[i].find_all('div', class_='a-row a-spacing-mini'): 
        try:   
            verified.append(tag.find('span', class_='a-size-small a-color-state profile-at-review-badge a-text-bold').get_text())
        except:
            continue  
        profile_urls.loc[i,'num_of_verified'] = len(verified)
    
    profile_urls.loc[i,'num_of_unverified'] = profile_urls.loc[i,'num_of_reviews_count'] - profile_urls.loc[i,'num_of_verified']
    
    date_mode_number = []
    # mode_number means if one person has many purchases on one day, how many purchases? I found out the date that appears most times.
    for tag in soup[i].find_all('div', class_='a-profile-content'):
        date_mode_number.append(tag.find('span', class_='a-profile-descriptor').get_text())
        profile_urls.loc[i,'mode_number'] = len([j for j, review in enumerate(date_mode_number) if review == max(set(date_mode_number), key=date_mode_number.count)])
    if profile_urls.loc[i,'mode_number'] > 20:
        profile_urls.loc[i,'samedate_20'] = 1
    else:
        profile_urls.loc[i,'samedate_20'] = 0
    
    

In [406]:
profile_urls

,product,profile,helpful_votes,name,num_of_reviews,num_of_reviews_count,num_of_verified,num_of_unverified,mode_number,samedate_20
0,B078N8NR7G,https://www.amazon.ca/gp/profile/amzn1.account...,1,Amazon Customer,4.0,4.0,4.0,0.0,3.0,0.0
1,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,0,nathalie,18.0,18.0,18.0,0.0,5.0,0.0
2,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,48,Conure Mum,161.0,120.0,113.0,7.0,13.0,0.0
3,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,0,Wayne Smith,2.0,2.0,2.0,0.0,1.0,0.0
4,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,3,Rob Self,7.0,7.0,7.0,0.0,5.0,0.0
5,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,5,Richard Goods,51.0,51.0,50.0,1.0,27.0,1.0
6,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,0,Jerome Tanguay,21.0,21.0,21.0,0.0,16.0,0.0
7,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,0,Tammy Roode,4.0,4.0,4.0,0.0,3.0,0.0
8,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,25,Lisa,98.0,98.0,95.0,3.0,12.0,0.0
9,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,0,mika jiang,5.0,5.0,5.0,0.0,4.0,0.0


In [414]:
product_df = pd.DataFrame(profile_urls['product'].unique(),columns = ['product'])

In [415]:
product_df 

,product
0,B078N8NR7G
1,B01HO8U5NC


In [425]:
for i in product_df.index.unique():
    product_df.loc[i, 'samedate_20'] = len(profile_urls[profile_urls['product'] == product_df.loc[i,'product']][profile_urls['samedate_20'] == 1])/len(profile_urls[profile_urls['product'] == product_df.loc[i,'product']])

/Users/jiaxuewan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [426]:
product_df

,product,samedate_20
0,B078N8NR7G,0.0
1,B01HO8U5NC,0.1


In [6]:
url=profile_urls.loc[0,'profile'] #The target url in short version.
user_agent = {'User-agent': 'Mozilla/5.0'} #Set up the user_agent.
response=requests.get(url,headers = user_agent) #Request the html.
file=open('test.txt','w') #Create a new file.
file.write(response.text) #Write the html requested into it.
file.close()

In [7]:
response.text

'\n\n \n \n\n\n\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!doctype html><html class="a-no-js" data-19ax5a9jf="dingo">\n  <head>\n<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>\n<script type="text/javascript">\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\nvar ue_hob=+new Date();\nvar ue_id=\'XATQQRH40PRC5MN8GPN8\',\nue_csm = window,\nue_err_chan = \'jserr-rw\',\nue = {};\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);\n\nue.stub(ue,"log");ue.stub(ue,"onunload");ue.stub(ue,"onflush");\n\n